# Verkenning stoplocaties AIS data

Met dit script identificeren we stoplocaties op zee met behulp van AIS data. 


### AIS data

Samenvattend bevat AIS data gegevens over de identiteit, positie, snelheid en koers van schepen. De volgende gegevens worden verzonden:
    AIS transmitters send data every 2 to 10 seconds while underway and every 3
    minutes while at anchor. The main AIS data field are:
        - MMSI (Maritime Mobile Service Identity) – a series of nine digits
          uniquely identifying ship stations;
        - Navigation status – “at anchor”, “under way using engine(s)”, or “not
          under command”;
        - Rate of turn – right or left, 0 to 720 degrees per minute;
        - Speed over ground – 0 to 102 knots with 0.1 knot resolution;
        - Position accuracy;
        - Longitude and Latitude – to 1/10,000 minute;
        - Course over ground – relative to true north to 0.1 degree;
        - True Heading – 0 to 359 degrees from gyro compass;
        - Timestamp – Coordinated Universal Time (UTC) time accurate to nearest
          second when this data was generated.
     
     Moreover, every 6 minutes the AIS transmitter sends additional fields,
     including:
         - IMO (International Maritime Organization) ship identification number
         - a seven digit number that remains unchanged upon transfer of the
           ship's registration to another Country;
         - International radio call sign – up to seven characters, assigned to
           the vessel by its Country of registry;
         - Vessel Name – 20 characters to represent the name of the vessel;
         - Type of ship/cargo;
         - Dimensions of ship – to nearest meter;
         - Type of positioning system – such as GPS, Differential Global
           Positioning Systems (DGPS) or Long Range Navigation (LORAN)-C;
         - Location of positioning system's antenna on-board the vessel;
         - Draught of ship – 0.1 meter to 25.5 meters;
         - Destination – max 20 characters;
         - Estimated time of arrival (ETA) at destination – UTC date hour:
           minute.

Bron: https://www.sciencedirect.com/science/article/pii/S2405535216300201

De reporting interval verhoogd als de snelheid van een schip verhoogd:
    - Position report:
        - 3 min: at anchor
        - 10 sec: < 14 knopen
        - 6 sec: < 23 knopen
    - Standard Class B equipment position report:
        - 3 min: < 2 knots
        - 30 sec: < 14 knots
        - 15 sec: < 23 knots

Bron: https://www.sciencedirect.com/science/article/pii/S0308597X17305535

Overige artikelen:
https://www.sciencedirect.com/science/article/pii/S0888613X13000728


### Stoplocaties vinden met DBScan

We zullen stoplocaties identificeren aan de hand van een model die het ID-Lab heeft gecreëerd om stoplocaties te vinden van taxichauffers(pitch 19.0002: Verkenning GPS data van taxi’s). Kort samengevat, identificeerd het model stoplocaties aan de geografische/ tijdsgegevens. Deze gegevens worden ingevoerd in de DBScan, die de bewegende en niet-bewegende punten clustert. Dit doet de DBScan aan de hand van een aangegeven epsilon en minimale aantal punten binnen de epsilon. Deze parameters worden in het model gekozen aan de hand van de Mann Withney-U test. Dan kiezen we die parameters, die de hoogste U waarde heeft op basis van gemiddelde snelheid. Dit houdt in: hoe hoger de u-waarde, hoe groter het verschil in snelheid tussen bewegende (ruis) en niet-bewegende punten (clusters).


## Aan de slag met AIS data van 1 schip

Het doel van deze opdracht is het identificeren van stoplocaties van schepen uit AIS data. Om goed zicht te hebben op wat we doen, analyseren we eerst data van 1 schip. We hebben een schip gekozen die veel te vinden is in de ankervakken. Dit zijn de schepen: 
    - Schip met MMSI 477050500 en IMO 248532000 
    - Schip met MMSI: 9605243 IMO 9417261 

Daarnaast is er een schip die actief vaart in de eerste miljoen rijen:
    - Schip met MMSI: 205370590


### Verkenning AIS data

Als eerst verkennen we de data door de coordinaten op een kaart te plotten en een visualiseren van de descriptive statistics. 

##### Stap 1: Importeer packages en data
Gezien het CSV bestand meer dan 500 miljoen rijen bevat, daarom itereren we door de rijen en verwijderen we alle rijen van andere schepen. Binnen de DAT-omgeving kunnen totaal 2 miljoen rijen worden opgeslagen via deze methode, daarna stopt de kernel wegens te veel RAM gebruik.

In [ ]:
#Import of different libraries
import matplotlib.pyplot as plt
plt.style.use('bmh')

import matplotlib.lines as mlines
from shapely.geometry import Polygon #Module for manipulation and analysis of geometric objects in the Cartesian plane.
import pandas as pd #This module provides high-performance, easy-to-use data structures and data analysis tools for Python
from shapely.geometry import Point #The Point constructor takes positional coordinate values or point tuple parameters to create a single point.
import numpy as np
from geopy import distance
import csv
import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
from tqdm import tqdm
import time
import os
import datetime
import fnmatch
import concurrent.futures
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import mpld3
import folium
from geopy.distance import geodesic
import seaborn as sns

###-----------------------------------------------------------------------------------------------
### HIERONDER DE METHODE OM DATA IN TE LEZEN. GEZIEN DEZE METHODE ERG LANG DUURT, HEB IK EEN NIEUWE CSV GEMAAKT. 

# with open("AIS_week.csv", newline='') as csvfile:
#     csvreader = csv.reader(csvfile);
#     header = next(csvreader);
#     data = {}
#     for h in header:
#         data[h] = []
#     for index, row in enumerate(csvreader):
# #         if index == 10000:
# #             break;
#         if row[5] == "477050500":
#             for h, v in zip(header, row):
#                 data[h].append(v);
                
# d = pd.DataFrame(data)
# d.to_csv("raw_MMSI_477050500.csv")
###---------------------------------------------------------------------------------------------------

# Inlezen van csv (hierboven gecreëerd)
df_raw = pd.read_csv("raw_MMSI_477050500.csv")

# Selecteren van de juiste kolommen
    # cols = list(df.columns.values)
df = df_raw[['t_starttime',
             't_updatetime',
             't_duration',
             't_mmsi',
             't_name',
             't_latitude',
             't_longitude',
             't_orientation',
             't_length',
             't_breadth',
             't_sensors',
             't_navstatus',
             't_imo',
             't_speed',
             't_heading',
             'p_destination',
             'p_draught',
             'p_antposfront',
             'p_antposleft',
             'p_shiptype',
             'p_cargotype']]


##### Stap 2: Plot coordinaten op kaart (Folium)

In [ ]:
# Zoek het middelste punt en zet de coordinaten in een lijst
mid_location = df['t_latitude'].mean(), df['t_longitude'].mean()
coords = df[["t_latitude", "t_longitude"]].values.tolist()
labels = df["t_name"].values.tolist()

# Creëer een folium map met de coordinaten
m = folium.Map(location=mid_location, zoom_start=8)
for point in range(len(coords)):
    popup = folium.Popup(labels[point], parse_html=True)
    folium.Marker(coords[point], popup=popup).add_to(m)

# Sla kaart op als HTML 
m.save('all_points_map_mmsi_477050500.html')

# Weergeef de kaart (functie doet het niet op de DAT-omgeving)
    #display(m)

### Data pre-processing 

##### Stap 1: omzetten van tijd en toevoegen van tijdskolommen

In [ ]:
# Omzetten van UTC tijd naar locale tijd (Amsterdam) en verwijderen overbodige kolommen
df["Time"] = pd.DatetimeIndex(df['t_updatetime']).time
df['Date'] = pd.DatetimeIndex(df['t_updatetime']).date
df['DateTime_UTC'] = df.apply(lambda r : pd.datetime.combine(r['Date'],r['Time']),1)
df['DateTime_Local'] = df['DateTime_UTC'].dt.tz_localize('utc').dt.tz_convert('Europe/Amsterdam')
del df['DateTime_UTC']
del df['Time']
del df['Date']

# splitten tijdgegevens in kolommen
df['Date'] = pd.DatetimeIndex(df['DateTime_Local']).date
df['Time'] = pd.DatetimeIndex(df['DateTime_Local']).time
df['DateTime'] = df["DateTime_Local"]

## Hieronder nog meer code om tijd op te splitten
df['Year'] = pd.DatetimeIndex(df['DateTime']).year
df['Month'] = pd.DatetimeIndex(df['DateTime']).month
df['Day'] = pd.DatetimeIndex(df['DateTime']).day
df['Weeknr'] = pd.DatetimeIndex(df['DateTime']).week
df['Weekdag'] = pd.DatetimeIndex(df['DateTime']).weekday
df['Hour'] = pd.DatetimeIndex(df['DateTime']).hour
df["minute"] = pd.DatetimeIndex(df["DateTime"]).minute
df["sec"] = pd.DatetimeIndex(df["DateTime"]).second

# Bereken datum in seconden
Year_sec = df["Year"] * 365 * 24 * 60 * 60
Month_sec = df["Month"] * 31 * 24 * 60 * 60
Day_sec = df["Day"] * 24 * 60 * 60
Hour_sec = df["Hour"] * 60 * 60
min_sec = df["minute"] * 60
sec_sec = df["sec"]


df["date_in_sec"] = Year_sec + Month_sec + Day_sec + Hour_sec + min_sec + sec_sec

#Verwijderen kolommen 
del df['DateTime_Local']
# del df["Dt"]

##### Stap 2: Afstand bereken tussen twee XY Punten

In [ ]:
# Nieuwe kolom (van a naar b) --> drop na 
df["Lat_b"] = df["t_latitude"].shift(-1)
df["Lon_b"] = df["t_longitude"].shift(-1)
df = df.dropna()

## Bereken afstand (van a naar b)
def distancer_km(row):
    coords_1 = (row['t_latitude'], row["t_longitude"])
    coords_2 = (row['Lat_b'], row['Lon_b'])
    return geodesic(coords_1, coords_2).km
    #return vincenty(coords_1, coords_2).km

def distancer_m(row):
    coords_1 = (row['t_latitude'], row['t_longitude'])
    coords_2 = (row['Lat_b'], row['Lon_b'])
    return geodesic(coords_1, coords_2).m
    #return vincenty(coords_1, coords_2).km

df['distance_km'] = df.apply(distancer_km, axis=1)
df['distance_m'] = df.apply(distancer_m, axis=1)

##### Stap 3:  Bereken tijd tussen twee XY punten

In [ ]:
##verschil in seconden tussen twee XY punten
df["date_b"] = df["DateTime"]
df["date_b"] = df["date_b"].shift(-1)
df = df.dropna(subset = ["DateTime", "date_b"]).reset_index(drop=True)
# df["date_b"] = df["date_b"].dropna()
df["diff"] = df["date_b"] - df["DateTime"]
df["diff_sec"] = df["diff"].astype('timedelta64[s]')

##### Stap 4: Bereken snelheid tussen twee XY Punten

In [ ]:
## meters per seconde / km per uur
df["speed_ms"] = df["distance_m"]/df["diff_sec"]
df["speed_kmu"] = df["distance_km"]/df["diff_sec"].divide(60*60)

##### Stap 5: Voeg tijdscomponent toe in volgnummer

In [ ]:
# Creer volgnummer met tijdselement
df["VgNr"] = df["diff_sec"].cumsum().astype(int)

##### Stap 6: Bereken aantal berichten per minuut

In [ ]:
# Aantal berichten per minuut
count_messages_per_minute = df.groupby(["Date", "minute"])["t_mmsi"].count().reset_index(name="messages_per_minute")
df_2 = pd.merge(df, count_messages_per_minute, left_on=["Date", "minute"], right_on=["Date", "minute"])

##### Stap 7: Sla pre-processed data op

In [ ]:
## Maak csv preprocessing
df_2.to_csv("AIS_preprocessing.csv")

### Descriptive statistics 

In [ ]:
# Heatmap dag vs tijd vs snelheid
x_snelheid_tijd = np.array(df["Time"])
y_snelheid_tijd = np.array(df["Date"])
z_snelheid_tijd = np.array(df["t_speed"])
results = pd.DataFrame.from_dict(np.array([x_snelheid_tijd,y_snelheid_tijd,z_snelheid_tijd]).T)
results.columns = ['x_snelheid_tijd','y_snelheid_tijd','z_snelheid_tijd']
pivotted = results.pivot('y_snelheid_tijd','x_snelheid_tijd','z_snelheid_tijd')

pivotted

sns.set()
pivotted.fillna(value=np.nan, inplace=True)

sns.heatmap(pivotted, cmap="RdBu")

In [ ]:
# Lijngrafiek datum vs snelheid vs navigatiestatus
sns.set(style="darkgrid")
sns.lineplot(x="DateTime", y="t_speed", hue="t_navstatus", data=df)

## Navigatiestatussen
# 0 = under way using engine
# 1 = at anchor
# 2 = not under command 
# 3 = restricted maneuverability
# 4 = constrained by her draught
# 5 = moored
# 6 = aground 
# 7 = engaged in fishing
# 8 = under way sailing
# 9 = reserved for future amendment of navigational status for ships carrying DG, HS, or MP, or IMO hazard or pollutant category C, high-speed craft (HSC)
# 10 = reserved for future amendment of navigational status for ships carrying dangerous goods (DG), harmful substances (HS) or marine pollutants (MP), or IMO hazard or pollutant category A, wing in ground (WIG)
# 11 = power-driven vessel towing astern (regional use)
# 12 = power-driven vessel pushing ahead or towing alongside (regional use)
# 13 = reserved for future use
# 14 = AIS-SART (active), MOB-AIS, EPIRB-AIS
# 15 = undefined = default (also used by AIS-SART, MOB-AIS and EPIRB-AIS under test)

In [ ]:
#Interactieve plot (datum vs snelheid) met Bokeh
from bokeh.plotting import figure, output_file, show

output_file("line.html")

p = figure(plot_width=1000, plot_height=1000)

# add a line renderer
p.line(df["DateTime"], df["t_speed"], line_width=2)

show(p)

In [ ]:
# Heatmap dag vs tijd vs snelheid
x_tijd_messages = np.array(df_2["Date"])
y_tijd_messages = np.array(df_2["Time"])
z_tijd_messages = np.array(df_2["messages_per_minute"])
results_tijd_message = pd.DataFrame.from_dict(np.array([x_tijd_messages, y_tijd_messages, z_tijd_messages]).T)
results_tijd_message.columns = ['x_tijd_messages', 'y_tijd_messages', 'z_tijd_messages']
pivotted_tijd_message = results_tijd_message.pivot('x_tijd_messages', 'y_tijd_messages', 'z_tijd_messages')
sns.set()
pivotted_tijd_message.fillna(value=np.nan, inplace=True)
sns.heatmap(pivotted_tijd_message, cmap="RdBu")

In [ ]:
# Lijngrafiek datum vs snelheid vs navigatiestatus
sns.set(style="darkgrid")
sns.lineplot(x="DateTime", y="messages_per_minute", hue="t_navstatus", data=df_2)

In [ ]:
#Interactieve plot (datum vs snelheid) met Bokeh
from bokeh.plotting import figure, output_file, show

output_file("line_messages_tijd.html")

p = figure(plot_width=1000, plot_height=1000)

# add a line renderer
p.line(df_2["DateTime"], df_2["messages_per_minute"], line_width=2)

show(p)

In [ ]:
# Lijngrafiek datum vs snelheid vs navigatiestatus
sns.set(style="darkgrid")
sns.lineplot(x="messages_per_minute", y="t_speed", data=df_2)

In [ ]:
# Plot speed vs messages per minute
y = df_2["t_speed"]
x = df_2["messages_per_minute"]

ax = sns.kdeplot(x, y)

In [ ]:
# Lijngrafiek messages per minuut vs orientation
sns.set(style="darkgrid")
sns.lineplot(x="messages_per_minute", y="t_orientation", data=df_2)

### Selecteren van parameters voor DBScan

##### Stap 1: inladen packages en data

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from sklearn import preprocessing

In [ ]:
df = pd.read_csv("AIS_preprocessing.csv")

##### Stap 2: Selecteren van variablen

In het algoritme van DBScan kun je variablen selecteren die de clusters gaan vormen. In deze opdracht zijn dat de lat/ lon, de snelheid en VgNr van een XY punt. 

In [ ]:
#Schalen van XYV punten 
coords_speed = np.asarray(df[['t_latitude', 't_longitude', "t_speed"]])
coords = preprocessing.scale(coords_speed)

coords

In [ ]:
# Functie maken om tabel te creëren met resultaten DBScan, u waarden en gemiddelde snelheden
def gemiddelde_kmu_per_eps(df, coords, eps, min_samples, speed_column="t_speed"):
    model = DBSCAN(eps=eps, min_samples=min_samples, algorithm="auto").fit(coords)
    cluster_labels = model.labels_
    num_clusters = len(set(cluster_labels))
    df["cluster"] = cluster_labels
    #selecteer snelheidskolom van clusters 
    clusters = df.t_speed[df["cluster"] > -1]
    #selecteer snelheidskolom van ruis
    ruis =df.t_speed[df["cluster"] == -1]
    #Verschil snelheidskolom van cluster vs ruis in u waarden
    u, prob = mannwhitneyu(clusters, ruis)
    kmu_per_cluster = df.groupby('cluster')["t_speed"].mean().reset_index(name='kmu_per_cluster')
    mean_kmu = kmu_per_cluster["kmu_per_cluster"].mean()
    return (eps, min_samples, num_clusters, mean_kmu, u)

# Bovenstaande functie uitvoeren voor 500 verschillende epsilons en 7 verschillende MinPoints
output_list = []
for epsilon in np.linspace(0.001, 0.05, 500):
    for min_samples in range(2, 20, 2):
        output = gemiddelde_kmu_per_eps(df=df, coords=coords, eps=epsilon, min_samples=min_samples)
        output_list.append(output)

# Resultaten functie naar tabel
output_df = pd.DataFrame(output_list, columns = ["eps", "min_samples", "NoP_cluster", "mean_kmu_speed", "MannWhitney_U"])

In [ ]:
# Creëer heatmap om zo de juiste parameters te kunnen selecteren. 
x = np.array(output_df["eps"])
y = np.array(output_df["min_samples"])
z = np.array(output_df["MannWhitney_U"])
results = pd.DataFrame.from_dict(np.array([x,y,z]).T)
results.columns = ['x','y','z']
pivotted = results.pivot('y','x','z')

import seaborn as sns
sns.set()

sns.heatmap(pivotted, cmap="RdBu")

In [ ]:
# Print per MinPoints de EPS met de hoogste u waarde

indices = results.groupby('y')['z'].idxmax; indices
u_waarden = results.loc[indices]
u_waarden["y"] = u_waarden["y"].astype(int)

print(u_waarden)


# results.to_csv("result_find_eps.csv")

In [ ]:
# from sklearn.metrics import average_precision_score 
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score

#Variabelen invoeren DBScan en schalen
df = pd.read_csv("AIS_preprocessing.csv")
coords_speed = np.asarray(df[['t_latitude', 't_longitude', "t_speed"]])
coords = preprocessing.scale(coords_speed)
eps_samples = u_waarden[["x", "y"]]

def dbs(df, coords, eps, min_samples):
    model = DBSCAN(eps=eps, min_samples=min_samples, algorithm="auto").fit(coords)
    cluster_labels = model.labels_
    num_clusters = len(set(cluster_labels))
    df["cluster"] = cluster_labels
    
    ##Bereken gemiddelde snelheid van het cluster
    kmu_per_cluster = df.groupby('cluster')['t_speed'].mean().reset_index(name='kmu_per_cluster')
    df = pd.merge(df, kmu_per_cluster, on=["cluster"], how="right")

    stoplocation = []
    for stop in df["kmu_per_cluster"]:
        if stop < 2:
            stoplocation.append(1)
        else:
            stoplocation.append(0)

    df["stoplocation"] = stoplocation
    
    #Voeg kolom toe die aangeeft wat een mogelijke stoplocatie is via navigatiestatus
    nav_stops = []
    for nav in df["t_navstatus"]:
        if nav == 1:
            nav_stops.append(1)
        else:
            nav_stops.append(0)

    df["nav_stops"] = nav_stops

    #precision van clusteren
    y_true = df["nav_stops"]
    y_pred = df["stoplocation"]
    
    #checken of de average="binary" de juiste methode is (sklearn)
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average="binary")
    count_nav = pd.value_counts(df['nav_stops'].values, sort=False)
    count_stop = pd.value_counts(df['stoplocation'].values, sort=False)  
    return (eps, min_samples, num_clusters, precision, recall, count_nav, count_stop)

#Voer dbscan uit + precision voor elke hoogste u waarden per min_samples
output_list_eps = []
for index, row in eps_samples.iterrows():
    output_eps = dbs(df=df, coords=coords, eps=row["x"], min_samples=row["y"])
    output_list_eps.append(output_eps)

# Uitkomst in dataframe + print dataframa
output_list_eps_df = pd.DataFrame(output_list_eps, columns = ["eps", "min_samples", "aantal_cluster", "percision", "recall", "count_nav", "count_stop"])
output_list_eps_df

### Cluster tabel maken
In deze stap zullen we de DBScan uitvoeren. Daarna zullen we de cluster verfijnen aan de hand van post-processing. In dit proces zullen we clusters die overlappen in tijd veranderen naar unieke clusters. Gezien binnen de tijden van 1 stoplocatie, niet een andere stoplocatie kan beginnen. Deze stoplocaties zijn uniek (Zie bokeh plot voor verdere uitleg). Daarna selecteren we van elk cluster het middelste XY punt. Aan dit XY Punt, kunnen we data linken zoals streetview foto's, CBS data, lijst met bedrijven, etc. Ook maken we voor de middelste XY punten een cluster tabel, zodat er een overzicht is per luster wat de gemiddelde snelheid is en het aantal punten per cluster.

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from sklearn import preprocessing
# from bokeh.plotting import figure, show, output_file
from bokeh.plotting import figure, output_file, show
from bokeh.io import push_notebook, show, output_notebook, curdoc, show
from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid
from bokeh.models.glyphs import VBar
output_notebook()

df = pd.read_csv("AIS_preprocessing.csv")
df_old = df #Nodig voor merge verderop in script

##### Stap 1: voer DBScan uit 

In [ ]:
#Variabelen invoeren DBScan en schalen
coords_speed = np.asarray(df[['t_latitude', 't_longitude', "t_speed"]])
coords = preprocessing.scale(coords_speed)

In [ ]:
# Selecteer beste parameters uit vorige stap (heatmap/ lijst beste parameters - U waarden)
# Uit de test hierboven blijkt epsilon 0.04 en min_samples 10 tot de juiste resultaten te leiden
epsilon = 0.043617
min_samples = 10

# run DBScan 
db = DBSCAN(eps=epsilon, min_samples=min_samples, algorithm='auto').fit(coords)
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
print('Number of clusters: {}'.format(num_clusters))

#Clusterlabels naar column in df
df["cluster"] = cluster_labels

In [ ]:
#merge met de oude gegevens 
df = df[["VgNr", "cluster"]]
df_new = pd.merge(df_old, df, on="VgNr", how="left")

# Opslaan raw data
df_new.to_csv("AIS_cluster_raw_eps0006_MinSamples2.csv")

##### Stap 2: Post-processing op clustervolgorde
In deze stap zorgen we ervoor dat de clusters alleen punten bevat die qua volgorde in tijd overeenkomen. 

In [ ]:
# maken van copy van df_total en sorteren op kolom clusters en tijd
Add_clusters = df_new.copy()
Add_clusters.sort_values(['VgNr'])
Add_clusters.head()

#Vinden van verschillen in kolom cluster_x
Add_clusters["Check_overlap"] = Add_clusters['cluster_x'].diff()
#Clusters nieuwe clusterwaarden geven als er een verschil is met vorige rij in kolom Check_overlap.
#Let op nieuwe clusternummers komen niet overeen met de oude clusternummers!
Add_clusters['Cluster_new']= (Add_clusters['Check_overlap'] != 0).astype(int).cumsum()

# De oorspronkelijke cluster -1 (=ruis) overnemen als None waarden in kolom Cluster_new
Add_clusters['Cluster_new'] = np.where(Add_clusters['cluster_x'] == -1, -1, (Add_clusters['Cluster_new']))

# Verwerken van de modelresultaten tot nieuwe clusters
result_total = Add_clusters.sort_values(['Cluster_new'])

df_vgnr = result_total.groupby('Cluster_new')['VgNr'].agg(['min','max']).rename(columns={'min': 'vgnr_min', 'max': 'vgnr_max'}).reset_index()
df_dt = result_total.groupby('Cluster_new')['DateTime'].agg(['min','max']).rename(columns={'min': 'dt_min', 'max': 'dt_max'}).reset_index()
df_concat = pd.merge(df_dt, df_vgnr, left_on='Cluster_new', right_on='Cluster_new')
Add_clusters["start"] = Add_clusters.DateTime.isin(df_concat.dt_min).astype(int)
Add_clusters["end"] = Add_clusters.DateTime.isin(df_concat.dt_max).astype(int)
Add_clusters["start"] = Add_clusters['start'].replace(0, np.nan)
Add_clusters["end"] = Add_clusters['end'].replace(0, np.nan)
Add_clusters.sort_values(['DateTime'], inplace=True)

In [ ]:
#Plot clusters op kaart

# Zoek het middelste punt en zet de coordinaten in een lijst
mid_location = Add_clusters['t_latitude'].mean(), Add_clusters['t_longitude'].mean()
coords = Add_clusters[["t_latitude", "t_longitude"]].values.tolist()
labels = Add_clusters["Cluster_new"].values.tolist()

# Creëer een folium map met de coordinaten
m = folium.Map(location=mid_location, zoom_start=8)
for point in range(len(coords)):
    popup = folium.Popup(labels[point], parse_html=True)
    folium.Marker(coords[point], popup=popup).add_to(m)

# Sla kaart op als HTML 
m.save('VgNr_mmsi_477050500.html')

# Weergeef de kaart (functie doet het niet op de DAT-omgeving)
    #display(m)

In [ ]:
# sla clusters op
Add_clusters.to_csv("raw_clusters.csv")

##### Stap 3: zoek middelste punt van cluster

In [ ]:
#Maak een array van lat/ lon en clusterslabels
coords_new = np.asarray(Add_clusters[['t_latitude', 't_longitude']])
cluster_array = np.asarray(Add_clusters["Cluster_new"])
num_cluster_array = len(set(cluster_array))
num_cluster_array_2 = 530
clusters = pd.Series([coords_new[cluster_array == n] for n in range(num_cluster_array_2)])
clusters = clusters[clusters.astype(str) != '[]']

#selecteer middelste XY punt
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)
centermost_points = clusters.map(get_centermost_point)

# maak dataframe
lats, lons = zip(*centermost_points)
rep_points = pd.DataFrame({'t_longitude':lons, 't_latitude':lats})

# print(pd.DataFrame(list(centermost_points))

##### Stap 4: Maak cluster tabel

In [ ]:
##Bereken gemiddelde snelheid en aantal XY punten per cluster
kmu_per_cluster = Add_clusters.groupby('Cluster_new')['t_speed'].mean().reset_index(name='kmu_per_cluster')
NoP_per_cluster = Add_clusters.groupby('Cluster_new')["Cluster_new"].count().reset_index(name='NoP_per_cluster')
distance_points_clusters = Add_clusters.groupby('Cluster_new')['distance_m'].mean().reset_index(name="distance_points_cluster")
NoP_kmu_per_cluster = pd.merge(kmu_per_cluster, NoP_per_cluster, on=["Cluster_new"], how="left")
NoP_kmu_distance_per_cluster = pd.merge(NoP_kmu_per_cluster, distance_points_clusters, on=["Cluster_new"], how="left")

## Voeg kolom toe die aangeeft of cluster een mogelijke stoplocatie is
all_cluster_table = pd.merge(Add_clusters, NoP_kmu_distance_per_cluster, on=["Cluster_new"], how="left")

stoplocation = []
for stop in all_cluster_table["kmu_per_cluster"]:
    if stop < 2:
        stoplocation.append(1)
    else:
        stoplocation.append(0)

all_cluster_table["stoplocation"] = stoplocation

# Maak een cluster tabel met alleen de middelste punten van een cluster
ID_centerpoint = all_cluster_table[["VgNr", "t_latitude", "t_longitude", "Lat_b", "Lon_b", "Cluster_new", "t_navstatus", "DateTime", "stoplocation"]]
ID_centerpoint_2 = pd.merge(rep_points, ID_centerpoint, on=["t_longitude", "t_latitude"], how="left")
cluster_table = pd.merge(ID_centerpoint_2, NoP_kmu_distance_per_cluster, on=["Cluster_new"], how="left")

# cluster tabel met alleen de stoplocaties
ct_stoplocaties = cluster_table[cluster_table["stoplocation"] ==1]

In [ ]:
# bereken accurancy

nav_stops = []
for nav in all_cluster_table["t_navstatus"]:
    if nav == 1:
        nav_stops.append(1)
    else:
        nav_stops.append(0)

all_cluster_table["nav_stops"] = nav_stops

y_score = all_cluster_table["nav_stops"]
y_test = all_cluster_table["stoplocation"]

from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, y_score)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

print(pd.value_counts(all_cluster_table['t_navstatus'].values, sort=False))
print(pd.value_counts(all_cluster_table['stoplocation'].values, sort=False))

In [ ]:
# Plot cluster tabel op kaart

# Zoek het middelste punt en zet de coordinaten in een lijst
mid_location = ct_stoplocaties['t_latitude'].mean(), ct_stoplocaties['t_longitude'].mean()
coords = ct_stoplocaties[["t_latitude", "t_longitude"]].values.tolist()
labels = ct_stoplocaties["VgNr"].values.tolist()

# Creëer een folium map met de coordinaten
m = folium.Map(location=mid_location, zoom_start=8)
for point in range(len(coords)):
    popup = folium.Popup(labels[point], parse_html=True)
    folium.Marker(coords[point], popup=popup).add_to(m)

# Sla kaart op als HTML 
m.save('cluster_table_stops.html')

# Weergeef de kaart (functie doet het niet op de DAT-omgeving)
    #display(m)

In [ ]:
# colors = [
#     'red',
#     'blue',
#     'gray',
#     'darkred',
#     'lightred',
#     'orange',
#     'beige',
#     'green',
#     'darkgreen',
#     'lightgreen',
#     'darkblue',
#     'lightblue',
#     'purple',
#     'darkpurple',
#     'pink',
#     'cadetblue',
#     'lightgray',
#     'black'
# ]

all_cluster_table["t_navstatus"] = all_cluster_table["t_navstatus"].astype(int)

colors = []
for status in all_cluster_table["t_navstatus"]:
    if status == 0:
        colors.append("green")
    elif status == 1:
        colors.append("red")
    elif status == 5:
        colors.append("orange")
    else:
        color.append("white")

all_cluster_table["colors"] = colors

# Zoek het middelste punt en zet de coordinaten in een lijst
mid_location = all_cluster_table['t_latitude'].mean(), all_cluster_table['t_longitude'].mean()
coords = all_cluster_table[["t_latitude", "t_longitude"]].values.tolist()
# labels = all_cluster_table["VgNr"].values.tolist()
labels = "ID:"+all_cluster_table["VgNr"].astype(str) +' '+all_cluster_table["t_name"]
colors = all_cluster_table["colors"].values.tolist()

# Creëer een folium map met de coordinaten
m = folium.Map(location=mid_location, zoom_start=8)
for point in range(len(coords)):
    popup = folium.Popup(labels[point], parse_html=True)
    folium.Marker(coords[point], popup=popup, icon=folium.Icon(color=colors[point])).add_to(m)

# Sla kaart op als HTML 
m.save('cluster_table_stops_color.html')

In [ ]:
#importing Pandas 
# https://medium.com/@ozan/interactive-plots-with-plotly-and-cufflinks-on-pandas-dataframes-af6f86f62d94
import pandas as pd
#importing plotly and cufflinks in offline mode
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

#we will get help from pivot tables to get Fare values in different columns for each class.
all_cluster_table[['t_speed', 'Cluster_new']].pivot(columns='Cluster_new', values='t_speed').iplot(kind='box')

In [ ]:
# #Maak boxplot
# %matplotlib notebook

# d_ata = {
#             "speed": Add_clusters["t_speed"],
#             "cluster_nummer": Add_clusters["Cluster_new"],
# }

# d_ata = pd.DataFrame(d_ata)

# d_ata[["speed", "cluster_nummer"]].boxplot( by="cluster_nummer", return_type="axes")

In [ ]:
# # from sklearn.metrics import average_precision_score 
# from sklearn.metrics import precision_score

# #Variabelen invoeren DBScan en schalen
# df = pd.read_csv("AIS_preprocessing.csv")
# coords_speed = np.asarray(df[['t_latitude', 't_longitude', "t_speed"]])
# coords = preprocessing.scale(coords_speed)
# eps_samples = u_waarden[["x", "y"]]

# def dbs(df, coords, eps, min_samples):
#     model = DBSCAN(eps=eps, min_samples=min_samples, algorithm="auto").fit(coords)
#     cluster_labels = model.labels_
#     num_clusters = len(set(cluster_labels))
#     df["cluster"] = cluster_labels
       
#     #Vinden van verschillen in kolom cluster_x
#     df["Check_overlap"] = df['cluster'].diff()
#     #Clusters nieuwe clusterwaarden geven als er een verschil is met vorige rij in kolom Check_overlap.
#     #Let op nieuwe clusternummers komen niet overeen met de oude clusternummers!
#     df['Cluster_new']= (df['Check_overlap'] != 0).astype(int).cumsum()

#     # De oorspronkelijke cluster -1 (=ruis) overnemen als None waarden in kolom Cluster_new
#     df['Cluster_new'] = np.where(df['cluster'] == -1, -1, (df['Cluster_new']))

#     # Verwerken van de modelresultaten tot nieuwe clusters
#     result_total = df.sort_values(['Cluster_new'])

#     df_vgnr = result_total.groupby('Cluster_new')['VgNr'].agg(['min','max']).rename(columns={'min': 'vgnr_min', 'max': 'vgnr_max'}).reset_index()
#     df_dt = result_total.groupby('Cluster_new')['DateTime'].agg(['min','max']).rename(columns={'min': 'dt_min', 'max': 'dt_max'}).reset_index()
#     df_concat = pd.merge(df_dt, df_vgnr, left_on='Cluster_new', right_on='Cluster_new')
#     df["start"] = df.DateTime.isin(df_concat.dt_min).astype(int)
#     df["end"] = df.DateTime.isin(df_concat.dt_max).astype(int)
#     df["start"] = df['start'].replace(0, np.nan)
#     df["end"] = df['end'].replace(0, np.nan)
#     df.sort_values(['DateTime'], inplace=True)
    
#     ##Bereken gemiddelde snelheid en aantal XY punten per cluster
#     kmu_per_cluster = df.groupby('Cluster_new')['t_speed'].mean().reset_index(name='kmu_per_cluster')
#     NoP_per_cluster = df.groupby('Cluster_new')["Cluster_new"].count().reset_index(name='NoP_per_cluster')
#     distance_points_clusters = df.groupby('Cluster_new')['distance_m'].mean().reset_index(name="distance_points_cluster")
#     NoP_kmu_per_cluster = pd.merge(kmu_per_cluster, NoP_per_cluster, on=["Cluster_new"], how="left")
#     NoP_kmu_distance_per_cluster = pd.merge(NoP_kmu_per_cluster, distance_points_clusters, on=["Cluster_new"], how="left")

#     ## Voeg kolom toe die aangeeft of cluster een mogelijke stoplocatie is
#     all_cluster_table = pd.merge(df, NoP_kmu_distance_per_cluster, on=["Cluster_new"], how="left")

#     stoplocation = []
#     for stop in all_cluster_table["kmu_per_cluster"]:
#         if stop < 2:
#             stoplocation.append(1)
#         else:
#             stoplocation.append(0)

#     all_cluster_table["stoplocation"] = stoplocation
    
#     #Voeg kolom toe die aangeeft wat een mogelijke stoplocatie is via navigatiestatus
#     nav_stops = []
#     for nav in all_cluster_table["t_navstatus"]:
#         if nav == 1:
#             nav_stops.append(1)
#         else:
#             nav_stops.append(0)

#     all_cluster_table["nav_stops"] = nav_stops

#     #precision van clusteren
#     y_true = all_cluster_table["nav_stops"]
#     y_pred = all_cluster_table["stoplocation"]

#     average_precision = precision_score(y_true, y_pred, average='binary')
#     count_nav = pd.value_counts(all_cluster_table['nav_stops'].values, sort=False)
#     count_stop = pd.value_counts(all_cluster_table['stoplocation'].values, sort=False)  
    
#     return (eps, min_samples, num_clusters, average_precision, count_nav, count_stop)

# #Voer dbscan uit + precision voor elke hoogste u waarden per min_samples
# output_list_eps = []
# for index, row in eps_samples.iterrows():
#     output_eps = dbs(df=df, coords=coords, eps=row["x"], min_samples=row["y"])
#     output_list_eps.append(output_eps)

# #Uitkomst in dataframe + print dataframa
# output_list_eps_df = pd.DataFrame(output_list_eps, columns = ["eps", "min_samples", "aantal_cluster", "percision", "count_nav", "count_stop"])
# output_list_eps_df

In [ ]:
# # speed vs cluster
# %matplotlib notebook

# y = d_ata["speed"]
# x = d_ata["cluster_nummer"]

# sns.set(style="darkgrid")
# sns.lineplot(x=x, y=y)

In [ ]:
# ct = cluster_table_no_1[cluster_table_no_1["kmu_per_cluster"] <=2]
# ct = cluster_table

# # %matplotlib notebook

# y = ct["kmu_per_cluster"]
# x = ct["Cluster_new"]

# sns.set(style="darkgrid")
# sns.lineplot(x=x, y=y)

In [ ]:
# %matplotlib notebook 
# # a = cluster_table_no_1["Cluster_new"]
# # b = cluster_table_no_1["kmu_per_cluster"]

# a = ct["Cluster_new"]
# b = ct["kmu_per_cluster"]
# # speed vs cluster
# sns.set(style="darkgrid")
# sns.lineplot(x=a, y=b)



In [ ]:
# # table: 
#     - mmsi 
#     - naam 
#     - shiptype
#     - t_starttime 
#     - destination
#     - tijd in dataset 
#     - aantal berichten 
#     - gemiddelde tijd tussen berichten 
#     - gemiddelde snelheid
#     - aantal punten at anchor / aantal punten niet at anchor
   
# Heatmap van snelheden vs tijd
# Heatmap van aantal berichten vs tijd
# Heatmap navstatus vs tijd
# Heatmap snelheid vs tijd
# heatmap aantal berichten vs navstatus

In [ ]:
# import matplotlib.pyplot as plt, mpld3


# x = Add_clusters["t_speed"]
# y = Add_clusters["cluster_x"]

# bp = plt.boxplot(x=x, labels=y)






# # p = BoxPlot(values=y, label=x,
# #             title="MPG Summary (grouped by CYL)")

# # output_file("boxplot.html")

# # show(p)



In [ ]:
# sns.set(style="ticks", palette="pastel")

# # Load the example tips dataset
# y = Add_clusters["t_speed"]
# x = Add_clusters["cluster_x"]

# # # Draw a nested boxplot to show bills by day and time
# sns.boxplot(x=x, y=y,)
# sns.despine(offset=10, trim=True)

In [ ]:
# Functie maken om tabel te creëren met resultaten DBScan, u waarden en gemiddelde snelheden
# def gemiddelde_distance_per_eps(df, coords, eps, min_samples, distance_column="distance_m"):
#     model = DBSCAN(eps=eps, min_samples=min_samples, algorithm="auto").fit(coords)
#     cluster_labels = model.labels_
#     num_clusters = len(set(cluster_labels))
#     df["cluster"] = cluster_labels
#     clusters = df.distance_m[df["cluster"] > -1]   
#     ruis =df.distance_m[df["cluster"] == -1]
#     u, prob = mannwhitneyu(clusters, ruis)
#     distance_per_cluster = df.groupby('cluster')["distance_m"].mean().reset_index(name='distance_per_cluster')
#     mean_distance = distance_per_cluster["distance_per_cluster"].mean()
#     return (eps, min_samples, num_clusters, mean_distance, u)

# # Bovenstaande functie uitvoeren voor 500 verschillende epsilons en 7 verschillende MinPoints
# output_list = []
# for epsilon in np.linspace(0.001, 0.05, 50):
#     for min_samples in range(1, 8, 1):
#         output = gemiddelde_distance_per_eps(df=df, coords=coords, eps=epsilon, min_samples=min_samples)
#         output_list.append(output)

# # Resultaten functie naar tabel
# output_df = pd.DataFrame(output_list, columns = ["eps", "min_samples", "NoP_cluster", "mean_distance", "MannWhitney_U"])

In [ ]:
# df_04_01 = df[(df[['Day']] == 1).all(axis=1)]

In [ ]:
# --> aantal berichten
# --> longest / smallest / mean time gap between messages
# --> van waar naar waar
# --> van hoelang hebben we data
# --> gemiddelde snelheid


In [ ]:
# data = [];
# with open("AIS_week.csv", newline='') as csvfile:
#     csvreader = csv.reader(csvfile);
#     for index, row in enumerate(csvreader):
#         if row[5] == "477050500":
#             data.append(tuple(row));

In [ ]:
# d = pd.DataFrame(data)
# d.to_csv("MMSI_477050500.csv")

In [ ]:
# df = pd.read_csv("MMSI_477050500.csv")

In [ ]:
# filename1 = '/data/candyreebroek/ID_lab/Schepen/Data/AIS_data_week.csv.zip'

In [ ]:
# data = [];
# with open("AIS_week.csv", newline='') as csvfile:
#     csvreader = csv.reader(csvfile);
# #     for i in tnrange(100, desc="loop"):
# #         sleep(0.01)
#     for index, row in enumerate(csvreader):
# #         if index == 1000000:
# #             break;
#         if row[5] == "477050500":
#             data.append(tuple(row));

In [ ]:
# data = [];
# with open("AIS_week.csv", newline='') as csvfile:
#     csvreader = csv.reader(csvfile);
#     values = range(3);
# #     with tqdm(total=len(values)) as pbar:
# #         for i in values:
# #             pbar.write('processed: %d' %i);
# #             pbar.update(1);
# #             sleep(1);
#         for index, row in enumerate(csvreader):
#             if index == 100000000:
#                 break;
#             if row[6] =="9605243":
#                 data.append(tuple(row));
        

#     for i in tqdm:
#         time.sleep(0.25);
#         pbar.set_description("Processing %s" % char);
#         time.sleep(0.01);
     

In [ ]:
# data = [];
# with open("AIS_week.csv", newline='') as csvfile:
#     csvreader = csv.reader(csvfile);
#     for i in tnrange(1, desc='1st loop'):
#         for j in tqdm_notebook(range(100), desc='2nd loop'):
#             sleep(0.01);
#         for index, row in enumerate(csvreader):
#             if index == 1000000:
#                 break;
#             if row[6] !="":
#                 data.append(tuple(row));

In [ ]:
# data = [];
# with open("AIS_week.csv", newline='') as csvfile:
#     csvreader = csv.reader(csvfile);
# #     next(csvreader); # Sla header over.
# #     for row in tqdm(csvreader):
#     for index, row in enumerate(csvreader):
#         if index == 3000000:
#             break;
#         if row[6] !="": 
#             data.append(tuple(row));

In [ ]:
# %%time

# # 1.000.000 is haalbaar


# df = pd.read_csv(filename1, nrows=2000000)

In [ ]:
# # Selecteer meest actieve schip
# df["count_name"] = df.groupby("t_mmsi")["t_mmsi"].transform("count")

# df.sort_values(["count_name"], ascending=False)




In [ ]:
# df.memory_usage().sum()

In [ ]:
# %%time
# for df in pd.read_csv(filename1, iterator=True, chunksize=1e8):
#     # Creation of a new dataframe df_new without the less relevant columns
#     systemID = df.iloc[:,0:1]
#     name_lat_long = df.iloc[:,6:9]
#     imo = df.iloc[:,33:34]
#     t_init = df.iloc[:,1:2]
#     t_update = df.iloc[:,2:3]
#     mmsi = df.iloc[:,5:6]
#     ves = df.iloc[:,40:41]
#     des = df.iloc[:,30:31]
#     speed = df.iloc[:,20:21]
#     heading = df.iloc[:,21:22]
#     length = df.iloc[:,11:12]
#     navstatus = df.iloc[:,14:15]

#     df_new = pd.concat([systemID, name_lat_long, imo, mmsi, ves, des, speed, heading, length, navstatus ,t_init, t_update, ],axis=1)
#     df_new.columns = ['SystemID', 'Name', 'Lat', 'Lon', 'IMO', 'MMSI', 'Schiptype', 'Destination', 'Speed', 'Heading', 'Length', "NavStatus", 'Starttime', 'Updatetime']